# import Dataset

In [1]:
A = open("C:/Users/Dell/Downloads/sherlock.txt",encoding="UTF-8")
data = A.read()

In [40]:
#data

# Tokenize data and  preprocessing

In [3]:
Q = []
import re
for i in data.split(" "):
    w = re.sub("[^a-zA-Z]","",i)
    w = w.upper()
    if(w!=""):
        Q.append(w)

In [4]:
Q = Q[0:1000]

# Find Unique words

In [5]:
from numpy import unique
unique_words = unique(Q)

# Create X and Y in string format

In [6]:
len(Q)

1000

In [7]:
Xstr = []
Ystr = []
for i in range(4,len(Q),1):
    w1 = Q[i-4]
    w2 = Q[i-3]
    w3 = Q[i-2]
    w4 = Q[i-1]
    wy = Q[i]
    line = w1 + " " + w2 + " " + w3 + " " + w4
    Xstr.append(line)
    Ystr.append(wy)

In [8]:
Q[0:20]

['PROJECT',
 'GUTENBERGS',
 'THE',
 'ADVENTURES',
 'OF',
 'SHERLOCK',
 'HOLMES',
 'BY',
 'ARTHUR',
 'CONAN',
 'DOYLETHIS',
 'EBOOK',
 'IS',
 'FOR',
 'THE',
 'USE',
 'OF',
 'ANYONE',
 'ANYWHERE',
 'AT']

In [9]:
from pandas import DataFrame
DataFrame([Xstr,Ystr]).T

,0,1
0,PROJECT GUTENBERGS THE ADVENTURES,OF
1,GUTENBERGS THE ADVENTURES OF,SHERLOCK
2,THE ADVENTURES OF SHERLOCK,HOLMES
3,ADVENTURES OF SHERLOCK HOLMES,BY
4,OF SHERLOCK HOLMES BY,ARTHUR
...,...,...
991,BY SIX ALMOST PARALLEL,CUTS
992,SIX ALMOST PARALLEL CUTS,OBVIOUSLY
993,ALMOST PARALLEL CUTS OBVIOUSLY,THEY
994,PARALLEL CUTS OBVIOUSLY THEY,HAVE


# Declare Arrays for X and Y

In [10]:
lines = len(Xstr)
n_uniques = len(unique_words)
seq = 4

In [11]:
from numpy import zeros
X_arr = zeros((lines,n_uniques,seq),dtype='uint8') 
Y_arr = zeros((lines,n_uniques)) 

# Data Preparation for X_arr and Y_arr

In [12]:
line_enum = list(enumerate(Xstr))

In [13]:
pos_finder = {}
for i,j in list(enumerate(unique_words)):
    pos_finder[j] = i

In [14]:
pos_finder["SHERLOCK"]

395

In [15]:
for line_no,line in line_enum:
    for seq,word in enumerate(line.split(" ")):
        X_arr[line_no,pos_finder[word],seq] = 1

In [31]:
#X_arr[3,11361,2]

In [17]:
Y_arr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [18]:
for i,j in enumerate(Ystr):
    pos = pos_finder[j]
    Y_arr[i,pos]=1

In [19]:
Y_arr

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [20]:
pos_finder["SHERLOCK"]

395

In [21]:
#Y_arr[1,11361]

In [22]:
X_arr.shape

(996, 515, 4)

# Create a NN

In [23]:
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.activations import softmax
from keras.layers.core import Activation

In [24]:
nn=Sequential()
nn.add(LSTM(200,input_shape=(515,4)))
nn.add(Dense(515))
nn.add(Activation("softmax"))

In [32]:
nn.compile(loss="categorical_crossentropy",metrics=["accuracy"],optimizer="adam")
model=nn.fit(X_arr,Y_arr,epochs=5,validation_split=0.01)

Epoch 1/5
31/31 [==============================] - 37s 1s/step - loss: 5.7690 - accuracy: 0.0578 - val_loss: 7.2196 - val_accuracy: 0.0000e+00
Epoch 2/5
31/31 [==============================] - 34s 1s/step - loss: 5.6415 - accuracy: 0.0578 - val_loss: 8.0390 - val_accuracy: 0.0000e+00
Epoch 3/5
31/31 [==============================] - 34s 1s/step - loss: 5.6110 - accuracy: 0.0578 - val_loss: 8.2602 - val_accuracy: 0.0000e+00
Epoch 4/5
31/31 [==============================] - 35s 1s/step - loss: 5.6014 - accuracy: 0.0578 - val_loss: 8.4227 - val_accuracy: 0.0000e+00
Epoch 5/5
31/31 [==============================] - 35s 1s/step - loss: 5.5977 - accuracy: 0.0578 - val_loss: 8.5219 - val_accuracy: 0.0000e+00


In [34]:
q = input("Enter sentence:(4 words)")
q = q.upper()

Enter sentence:(4 words)adventures of sherlock holmes


In [35]:
q

'ADVENTURES OF SHERLOCK HOLMES'

In [36]:
inp = zeros((1,515,4),dtype="bool")

for word_no,word in enumerate(q.split(" ")):
    inp[0,pos_finder[word],word_no]=1

In [37]:
import pandas as pd
B = pd.DataFrame(zip(unique_words,list(nn.predict(inp)[0])),columns=["Word","Prob"])
x = B.Prob.max()

In [38]:
B[B.Prob == x].Word.values[0]

'THE'